In [25]:
import requests
from bs4 import BeautifulSoup
import json
import concurrent.futures
import re
import queue

In [26]:
_phuongnam_url = "https://nhasachphuongnam.com/vi/sach-tieng-viet-page-{}.html?items_per_page=128&result_ids=pagination_contents&is_ajax=1"
headers = {
  'Connection': 'keep-alive',
  'Pragma': 'no-cache',
  'Cache-Control': 'no-cache',
  'sec-ch-ua': '"Chromium";v="94", "Google Chrome";v="94", ";Not A Brand";v="99"',
  'Accept': 'application/json, text/javascript, */*; q=0.01',
  'X-Requested-With': 'XMLHttpRequest',
  'sec-ch-ua-mobile': '?0',
  'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.71 Safari/537.36',
  'sec-ch-ua-platform': '"Windows"',
  'Sec-Fetch-Site': 'same-origin',
  'Sec-Fetch-Mode': 'cors',
  'Sec-Fetch-Dest': 'empty',
  'Accept-Language': 'vi,en-US;q=0.9,en;q=0.8,vi-VN;q=0.7',
  'Cookie': 'sid_customer_d577e=c81a4f54e12eb374c53db1cf4d78f94e-C'
}

response = requests.request("GET", url, headers=headers, verify=False).json()

C:\Users\Watch Dogs\Anaconda3\envs\books\lib\site-packages\urllib3\connectionpool.py:1020: InsecureRequestWarning: Unverified HTTPS request is being made to host 'nhasachphuongnam.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


In [8]:
response.keys()

dict_keys(['current_url', 'html', 'notifications'])

In [12]:
html = BeautifulSoup(response['html']['pagination_contents'])

In [24]:
book_html=  html.find('div',{'class':'grid-list vs-grid-table-wrapper et-grid-table-wrapper'})
bookshelf = book_html.findAll('div',{'class':'vs-grid vs-grid-table et-grid'})
bookshelf[0].a['href']

'https://nhasachphuongnam.com/vi/bai-tap-tieng-anh-theo-chuong-trinh-moi-cua-bo-gddt-lop-9-tap-2-co-dap-an.html'

In [ ]:
booksQueue = queue.Queue()

class _phuongnam_Crawler():
    def __init__(self, categoryid, urlkey):
        self.categoryid = categoryid
        self.urlkey = urlkey
        
    def crawl(self):
        _response = requests.request("GET", _category_path.format(self.categoryid, 1, self.urlkey), headers=headers, data=payload).json()['paging']['last_page']
        print(f'total pages {_response}')
        _concurrent = [_ for _ in range(_response+1)]
        print(_concurrent)
#         _concurrent = [_ for _ in range(10)]
        
        with concurrent.futures.ThreadPoolExecutor(max_workers=20) as executor:
            executor.map(self._products, _concurrent)
    
    def _products(self, pages):
        _res = requests.request("GET", _category_path.format(self.categoryid, pages, self.urlkey), headers=headers, data=payload).json()
        
        nums = len(_res['data'])
        print(f"total books of {pages} is ",nums)
        for _ in _res['data']:
            book = {}
            
            book['Tên Sách'] = _['name']
            book['Tên tác giả'] = _['author_name']
            book['Thể loại'] = _reverse_mapping[self.categoryid]
            book['Nội dung tóm tắt'] = _['short_description']
            book['Giá bìa'] = _['original_price']
            
            booksQueue.put(book)
            
        print(f"Number of products in my pocket {booksQueue.qsize()}")
        
    def _bookshelf(self):
        response = requests.request("GET", url, headers=headers, verify=False).json()
        
        html = BeautifulSoup(response['html']['pagination_contents'])
        book_html=  html.find('div',{'class':'grid-list vs-grid-table-wrapper et-grid-table-wrapper'})
        books = book_html.findAll('div',{'class':'vs-grid vs-grid-table et-grid'})
        
        _bookshelf = [_a['href'] for _ in books]
        return _bookshelf